# import libraries

In [1]:
import wandb
from models.ResNet_model import *
#from models.{team_name}_model import *
from utils import *
import h5py
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

/home/muntakimrafi/anaconda3/envs/dream_autosome/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# read data

In [2]:
h5f = h5py.File('X_train.h5','r')
X_train = h5f['X_train'][:]
h5f.close()
h5f = h5py.File('X_val.h5','r')
X_val = h5f['X_val'][:]
h5f.close()
h5f = h5py.File('y_train.h5','r')
y_train = h5f['y_train'][:]
h5f.close()
h5f = h5py.File('y_val.h5','r')
y_val = h5f['y_val'][:]
h5f.close()

In [3]:
print(X_train.shape)
print(X_val.shape)
print(y_train.shape)
print(y_val.shape)

(6065331, 4, 110)
(673926, 4, 110)
(6065331,)
(673926,)


# create dataLoader

In [4]:
trainLoader = createDataLoader(X_train, y_train, num_workers=0, batch_size=1024, shuffle=True, drop_last=True, 
                           random_seed = 42)
valLoader = createDataLoader(X_val, y_val, num_workers=0, batch_size=1024, shuffle=True, drop_last=True, random_seed = 42)

# define all the possible options for different layers that your model can accept

In [5]:
l1s = [FirstLayers(4, 64, 7, 1, 'same'), ]
l2s = [ResidualLayers(ResidualBlock, [3,4,6,3]), ]
l3s = [FinalLayers(14, 512, 1), ]
lossFunctions = [torch.nn.MSELoss().to(device),]

# choose a specific combination from all possible options

##### after you finish defining l1s, l2s, ... ... ..., lossFunctions, lrs for your model you can just run a for loop to create all possible model combinations

In [6]:
layers = [l1s[0], l2s[0], l3s[0]]

model = PrixFixeNet(layers).to(device)

In [7]:
# torchsummary may not work for your model
summary(model, (4, 110))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv1d-1              [-1, 64, 110]           1,856
       BatchNorm1d-2              [-1, 64, 110]             128
              ReLU-3              [-1, 64, 110]               0
       FirstLayers-4              [-1, 64, 110]               0
            Conv1d-5              [-1, 64, 110]          12,352
       BatchNorm1d-6              [-1, 64, 110]             128
              ReLU-7              [-1, 64, 110]               0
            Conv1d-8              [-1, 64, 110]          12,352
       BatchNorm1d-9              [-1, 64, 110]             128
             ReLU-10              [-1, 64, 110]               0
    ResidualBlock-11              [-1, 64, 110]               0
           Conv1d-12              [-1, 64, 110]          12,352
      BatchNorm1d-13              [-1, 64, 110]             128
             ReLU-14              [-1, 

In [10]:
loss_fn = lossFunctions[0]
optimizer = torch.optim.Adam(model.parameters(), lr=3e-4)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', factor=0.1, patience=10)

In [11]:
#each combination will be assinged an ID to keep track of the combinations being tested.
modelID = 0

#train the model using this sample trainer function
train(trainLoader, model, loss_fn, optimizer, scheduler, valLoader, 10, modelID)

Epoch 1, 92160 sequences, time: 8.828070402145386s, train Loss: 3.1123338646358913, train mse 3.1123338487413195, train pearson: 0.6662337454961247, train_spearman: 0.6820760031011348, val Loss: 3.0073201656341553, val mse: 3.007320022583008, val pearson: 0.6787708088781681, val spearman: 0.6938471647957187
Epoch 2, 92160 sequences, time: 8.974015235900879s, train Loss: 2.985418004459805, train mse 2.985418023003472, train pearson: 0.6828709163824257, train_spearman: 0.6977301181554028, val Loss: 2.9682958364486693, val mse: 2.9682960510253906, val pearson: 0.6890987312567575, val spearman: 0.7044407691025286
Epoch 3, 92160 sequences, time: 8.79938530921936s, train Loss: 2.9942171732584635, train mse 2.994216579861111, train pearson: 0.68404725313188, train_spearman: 0.6997012274911651, val Loss: 2.979528522491455, val mse: 2.9795284271240234, val pearson: 0.6900608283058259, val spearman: 0.7028745819875751
Epoch 4, 92160 sequences, time: 8.828262090682983s, train Loss: 2.942710812886

PrixFixeNet(
  (l1): FirstLayers(
    (conv1): Sequential(
      (0): Conv1d(4, 64, kernel_size=(7,), stride=(1,), padding=same)
      (1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
    )
  )
  (l2): ResidualLayers(
    (layer0): Sequential(
      (0): ResidualBlock(
        (conv1): Sequential(
          (0): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(1,))
          (1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU()
        )
        (conv2): Sequential(
          (0): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(1,))
          (1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
        (relu): ReLU()
      )
      (1): ResidualBlock(
        (conv1): Sequential(
          (0): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(1,))
          (1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_

In [2]:
import numpy as np

a = np.array([1,2])
b = np.array([3,4])
c = [a,b]
d = np.array(c)
print(d)
print(d.shape)

[[1 2]
 [3 4]]
(2, 2)
